<b>Spark Streaming para Elasticsearch</b>

In [1]:
import os
import urllib.request
import zipfile
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import json
import time
from datetime import datetime

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es = Elasticsearch('elasticsearch-node:9200')

if es.indices.exists('stream-test'):
    es.indices.delete('stream-test')
    
    body={
        'mappings': {
            'properties': {
                'count': {'type': 'text'},
                'name': {'type': 'text'},
                'value': {'type': 'text'},
                'timestamp': {'type': 'text'}
            }
        }
    }
    
    es.indices.create(index='stream-test', body=body)
    
print(es.info())

{'name': '927d8d22d8c7', 'cluster_name': 'docker-cluster', 'cluster_uuid': '038uq0CNScCWYvn2HwLEoA', 'version': {'number': '7.5.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '8bec50e1e0ad29dad5653712cf3bb580cd1afcdf', 'build_date': '2020-01-15T12:11:52.313576Z', 'build_snapshot': False, 'lucene_version': '8.3.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars elasticsearch-hadoop-7.5.2/dist/elasticsearch-spark-20_2.11-7.5.2.jar pyspark-shell'
sc = SparkContext(appName="PythonSparkStreaming")
sc.setLogLevel("WARN")

In [5]:
ssc = StreamingContext(sc, 3)

In [20]:
print(os.getcwd())
json_folder_path = ('./sample/')

json_folder_path = ('./sample/')
json_files = [ x for x in os.listdir(json_folder_path ) if x.endswith("json") ]
json_merge = {}

for json_file in json_files:

    json_file_path = os.path.join(json_folder_path, json_file)

    with open (json_file_path) as f:

        data_dict = json.loads(f.read())  
        json_merge = json.dumps(data_dict)
        json_merge = str(json_merge)
        #json_merge.update(data_dict)   
        
        #print(json_file)
        #print(json_merge['count'])
        #teste = json.dumps(data_dict, indent=4)
        data_dict['count'] = data_dict.pop('count')
        print(json_merge)
        print(data_dict['count'])
        
        #-----------------------------------------------------------
        
        #'''
        stream = ssc.textFileStream(json_file_path)
        
        es_write_conf = {
            "es.nodes" : '927d8d22d8c7',
            "es.port" : '9200',
            "es.resource" : 'stream-test',
            "es.input.json" : "yes",
            "es.mapping.id": "count"
        }
        
        rdd = sc.parallelize(data_dict)
        
        def format_data(x):
            return (data_dict['count'], json.dumps(data_dict))      

        # Uma função lambda pode receber qualquer número de argumentos, 
        # mas pode ter apenas uma expressão.

        rdd = rdd.map(lambda x: format_data(x))
        
        parsed = stream.map(lambda x: format_data(x))
        
        rdd.saveAsNewAPIHadoopFile(
            path='-',
            outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
            keyClass="org.apache.hadoop.io.NullWritable",
            valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
            conf=es_write_conf
        )
        
        #'''

/home/jovyan
{"count": 5, "name": "Bilbo", "value": 37, "timestamp": 1581297174}
5
{"count": 3, "name": "Bilbo", "value": 66, "timestamp": 1581297172}
3
{"count": 4, "name": "Legolas", "value": 52, "timestamp": 1581297173}
4
{"count": 11, "name": "Frodo", "value": 4, "timestamp": 1581297180}
11
{"count": 7, "name": "Legolas", "value": 56, "timestamp": 1581297176}
7
{"count": 8, "name": "Aragorn", "value": 78, "timestamp": 1581297177}
8
{"count": 0, "name": "Legolas", "value": 54, "timestamp": 1581297169}
0
{"count": 6, "name": "Samwise", "value": 54, "timestamp": 1581297175}
6
{"count": 9, "name": "Frodo", "value": 91, "timestamp": 1581297178}
9
{"count": 2, "name": "Bilbo", "value": 79, "timestamp": 1581297171}
2
{"count": 12, "name": "Legolas", "value": 57, "timestamp": 1581297181}
12


In [21]:
#stream = ssc.textFileStream(json_file_path) 

In [22]:
'''
es_write_conf = {
    "es.nodes" : '927d8d22d8c7',
    "es.port" : '9200',
    "es.resource" : 'stream-test',
    "es.input.json" : "yes",
    "es.mapping.id": "count"
}
'''

'\nes_write_conf = {\n    "es.nodes" : \'927d8d22d8c7\',\n    "es.port" : \'9200\',\n    "es.resource" : \'stream-test\',\n    "es.input.json" : "yes",\n    "es.mapping.id": "count"\n}\n'

In [23]:
'''
rdd = sc.parallelize(json_merge)

print(json_merge)
'''

'\nrdd = sc.parallelize(json_merge)\n\nprint(json_merge)\n'

In [24]:
'''
def format_data(x):
    return (json_merge['count'], json.dumps(json_merge))      

# Uma função lambda pode receber qualquer número de argumentos, 
# mas pode ter apenas uma expressão.

rdd = rdd.map(lambda x: format_data(x))
'''

"\ndef format_data(x):\n    return (json_merge['count'], json.dumps(json_merge))      \n\n# Uma função lambda pode receber qualquer número de argumentos, \n# mas pode ter apenas uma expressão.\n\nrdd = rdd.map(lambda x: format_data(x))\n"

In [25]:
#parsed = stream.map(lambda x: format_data(x))

In [26]:
'''
rdd.saveAsNewAPIHadoopFile(
    path='-',
    outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
    keyClass="org.apache.hadoop.io.NullWritable",
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
    conf=es_write_conf
)
'''

'\nrdd.saveAsNewAPIHadoopFile(\n    path=\'-\',\n    outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",\n    keyClass="org.apache.hadoop.io.NullWritable",\n    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",\n    conf=es_write_conf\n)\n'

In [27]:
parsed.foreachRDD(lambda rdd: handler(rdd))

In [28]:
parsed.pprint()

In [29]:
ssc.start()

-------------------------------------------
Time: 2020-02-14 01:32:48
-------------------------------------------

-------------------------------------------
Time: 2020-02-14 01:32:51
-------------------------------------------

-------------------------------------------
Time: 2020-02-14 01:32:54
-------------------------------------------

-------------------------------------------
Time: 2020-02-14 01:32:57
-------------------------------------------



In [30]:
ssc.stop()

<b>Bulk Processing ES with Spark</b>

In [31]:
sc = SparkContext(appName="PythonSparkReading")  
sc.setLogLevel("WARN")

In [32]:
es_read_conf = {
    "es.nodes" : '927d8d22d8c7',
    "es.port" : '9200',
    "es.resource" : 'stream-test'
    }

In [33]:
es_rdd = sc.newAPIHadoopRDD(
    inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
    keyClass="org.apache.hadoop.io.NullWritable",
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
    conf=es_read_conf)

In [34]:
es_rdd.take(5)

[('4',
  {'name': 'Legolas', 'count': '4', 'value': '52', 'timestamp': '1581297173'}),
 ('8',
  {'name': 'Aragorn', 'count': '8', 'value': '78', 'timestamp': '1581297177'}),
 ('0',
  {'name': 'Legolas', 'count': '0', 'value': '54', 'timestamp': '1581297169'}),
 ('6',
  {'name': 'Samwise', 'count': '6', 'value': '54', 'timestamp': '1581297175'}),
 ('7',
  {'name': 'Legolas', 'count': '7', 'value': '56', 'timestamp': '1581297176'})]

In [35]:
es_rdd = es_rdd.map(lambda x: x[1])

In [36]:
es_rdd.take(1)

[{'name': 'Legolas', 'count': '4', 'value': '52', 'timestamp': '1581297173'}]

In [37]:
from pyspark.sql import SparkSession, SQLContext, Row

spark = SparkSession \
    .builder \
    .appName("Spark SQL") \
    .getOrCreate()

In [38]:
df = es_rdd.map(lambda l: Row(**dict(l))).toDF()

In [39]:
df.take(1)

[Row(count='4', name='Legolas', timestamp='1581297173', value='52')]

In [40]:
df \
    .groupby('value') \
    .count() \
    .collect()

[Row(value='54', count=2),
 Row(value='52', count=1),
 Row(value='78', count=1),
 Row(value='79', count=1),
 Row(value='56', count=1),
 Row(value='37', count=1),
 Row(value='4', count=1),
 Row(value='66', count=1),
 Row(value='91', count=1),
 Row(value='57', count=1)]

In [44]:
df \
    .filter(df.name == 'Legolas')\
    .take(5)

[Row(count='4', name='Legolas', timestamp='1581297173', value='52'),
 Row(count='0', name='Legolas', timestamp='1581297169', value='54'),
 Row(count='7', name='Legolas', timestamp='1581297176', value='56'),
 Row(count='12', name='Legolas', timestamp='1581297181', value='57')]

In [45]:
sc.stop()